In [1]:
import numpy as np
import pandas as pd
import glob
import os, sys
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier,
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
VERSION = 4
RANDOM_SEED = 7

## Load data

### Load Data - downsampled

In [2]:
# df_joint_train_org   = pd.read_csv(f'./features/cache_all_features_train_V{VERSION}.csv')
# df_joint_train_org   = df_joint_train_org.drop(columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])
                       
# df_joint_test_org = pd.read_csv(f'./features/cache_all_features_test_V{VERSION}.csv').drop(
#     columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])

# print("shape of train set: ", df_joint_train_org.shape)
# print("shape of test  set: ", df_joint_test_org.shape)


# df_joint_train  = pd.read_csv(f'./features/cache_train_V4_resampled_2500.csv')
df_joint_test = pd.read_csv(f'./features/cache_test_V4_resampled_250.csv')

# print("shape of train set: ", df_joint_train.shape)
print("shape of test  set: ", df_joint_test.shape)

df_joint_train_aug  = pd.read_csv(f'./features/cache_train_V4_augmented.csv')
feature_column_names = [i for i in df_joint_train_aug.columns \
                        if i not in ['file_path','renamed_file_path','split','sentiment_value','emotional_category']]
             
print("shape of train set: ", df_joint_train_aug.shape)
df_joint_train_aug.groupby('sentiment_value')['file_path'].count()

shape of test  set:  (680, 1546)
shape of train set:  (24885, 1546)


sentiment_value
-1    7999
 0    8560
 1    8326
Name: file_path, dtype: int64

### best guess feature combinations

In [3]:
# generate selected features 
def generate_selected_features_by_type(feature_column_names,input,stats,number=1):
    selected_result = []
    for name in feature_column_names:
        if input+"_"+stats in name:
            selected_result.append(name)
    if number < len(selected_result):
        selected_result = selected_result[:number]
    return selected_result

# example to take mfcc 20 mean & std; mel32; zcr all 5 stats features
feature_MFCC20_mean  = generate_selected_features_by_type(feature_column_names,"mfcc","mean",20)
feature_MFCC20_std   = generate_selected_features_by_type(feature_column_names,"mfcc","std",20)
feature_mel32_median = generate_selected_features_by_type(feature_column_names,"mel32","median",32)
feature_mel32_std    = generate_selected_features_by_type(feature_column_names,"mel32","std",32)
feature_zcr_stats    = generate_selected_features_by_type(feature_column_names,"zcr","",5)
feature_rms_stats    = generate_selected_features_by_type(feature_column_names,"rms","",5)
selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum','Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
selected_prosody = selected_intensity + selected_pitch # + ['Local Jitter','Local Shimmer']
selected_feature_names = feature_MFCC20_mean + feature_MFCC20_std + feature_mel32_median + feature_mel32_std + \
                        feature_zcr_stats + feature_rms_stats + selected_intensity + selected_pitch 


In [4]:
# default use augmented training set and balanced test set
X_train = df_joint_train_aug[selected_feature_names]
y_train_s = df_joint_train_aug['sentiment_value']
y_train_e = df_joint_train_aug['emotional_category']

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y_e_num = label_encoder.fit_transform(y_train_e)

X_test = df_joint_test[selected_feature_names]
y_test_s = df_joint_test['sentiment_value']
y_test_e = df_joint_test['emotional_category']

# y_test_e_num = label_encoder.fit_transform(y_test_e)

In [5]:
X_train.shape, X_test.shape

((24885, 128), (680, 128))

## Models

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier,HistGradientBoostingClassifier


# Common adjustable parameters
common_params = {
    'RandomForest': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'SVM': {'kernel': 'rbf', 'C': 1.0, 'probability': True},
    'KNN': {'n_neighbors': 5},
    'GradientBoosting': {'loss': 'log_loss', 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0,
                         'criterion': 'friedman_mse', 'min_samples_split': 2, 'max_depth': 3},
    'GradientBoostingFast': {'loss': 'log_loss', 'learning_rate': 0.1, 'max_iter': 100},
    'AdaBoost': {'n_estimators': 50, 'learning_rate': 1.0},
    'LightGBM': {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0,
                 'min_child_samples': 20, 'max_depth': -1}    
}

# Models with common adjustable parameters
models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(**common_params['RandomForest']),
    'SVM': SVC(**common_params['SVM']),
    'KNN': KNeighborsClassifier(**common_params['KNN']),
    'GradientBoosting': GradientBoostingClassifier(**common_params['GradientBoosting']),
    'GradientBoostingFast': HistGradientBoostingClassifier(**common_params['GradientBoostingFast']),
    'AdaBoost': AdaBoostClassifier(**common_params['AdaBoost']),
    # 'LightGBM': LGBMClassifier(**common_params['LightGBM'])
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3364612971.py, line 30)

In [7]:
def try_clf_with_feature_selected(clf_name, X_train, X_test, y_train, y_test):
    start = time.time()
    print(f'Model Name: {clf_name};\n Train set shape {X_train.shape}, num of class {y_train.unique().size}')
    clf_model = models[clf_name]
    if clf_name == 'KNN':
        predictions = clf_model.fit(X_train, y_train).predict(X_test.values)
    else:
        predictions = clf_model.fit(X_train, y_train).predict(X_test)
    
    print(classification_report(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    
    precision, recall, f1score, support = precision_recall_fscore_support(y_test_s, predictions, average=None)
    # TODO make all metrics into result dict for recording 
    probabilities = clf_model.predict_proba(X_test) if clf_name != 'KNN' else  clf_model.predict_proba(X_test.values)
    print('prbabilities distribution: \n', pd.DataFrame(probabilities,columns=clf_model.classes_).describe())
    print(f'Time taken: {round(time.time()-start,3)} seconds.\n')

### Sentiment 3-class Random Forest Classifier

In [8]:
for model_name, model in models.items():
    try_clf_with_feature_selected(model_name, X_train, X_test, y_train_s, y_test_s)

Model Name: DecisionTree;
 Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.57      0.70      0.63       248
           0       0.70      0.59      0.64       183
           1       0.69      0.61      0.65       249

    accuracy                           0.64       680
   macro avg       0.65      0.63      0.64       680
weighted avg       0.65      0.64      0.64       680

[[174  29  45]
 [ 52 108  23]
 [ 80  17 152]]
prbabilities distribution: 
               -1           0           1
count  680.00000  680.000000  680.000000
mean     0.45000    0.226471    0.323529
std      0.49786    0.418855    0.468167
min      0.00000    0.000000    0.000000
25%      0.00000    0.000000    0.000000
50%      0.00000    0.000000    0.000000
75%      1.00000    0.000000    1.000000
max      1.00000    1.000000    1.000000
Time taken: 3.564 seconds.

Model Name: RandomForest;
 Train set shape (24885, 128), num of class 3
   

In [ ]:
try_clf_with_feature_selected(model_name, X_train, X_test, y_train_e, y_test_e)

### ChatGPT

### Threshold tuning

In [ ]:
# Calculate accuracy for the given threshold
def calc_acc_by_thres(probabilities, threshold, y_test):
    predictions_adj = []
    # Loop through each sample's probabilities
    for probs in probabilities:
        if probs[0] > threshold:
            pred_class = -1
        elif probs[1] > probs[2]:
            pred_class = 0
        else:
            pred_class = 1
        predictions_adj.append(pred_class)
    accuracy = np.mean(predictions_adj == y_test)
    precision, recall, f1score, _ = precision_recall_fscore_support(y_test_s, predictions_adj, average=None)
    return accuracy, min(f1score), np.var(f1score)

best_threshold = None
best_accuracy = 0.0
best_f1score = 0.0
# best_f1s_var = 10

# Define a range of threshold values to try
threshold_range = np.linspace(0.25, 0.75, 100)
for threshold in threshold_range:
    accuracy, min_f1_score, var_f1_score = calc_acc_by_thres(probabilities, threshold, y_test_s)
    # if accuracy > best_accuracy:
    #     best_accuracy = accuracy
    #     best_threshold = threshold
    if min_f1_score > best_f1score:
        best_f1score = min_f1_score
        best_threshold = threshold
        best_accuracy = accuracy
    # if var_f1_score < best_f1s_var:
    #     best_f1score = min_f1_score
    #     best_threshold = threshold
    #     best_accuracy = accuracy
    #     best_f1s_var = var_f1_score
        

print("Best Threshold:", best_threshold)
print("Best Accuracy:", best_accuracy)
print("Best min f1 score:", best_f1score)
# print("Best var f1 score:", best_f1s_var)

To get overall high accuracy: Best Threshold: 0.5328 ; Best Accuracy: 0.7746   
To get 0 & 1 high f1 score to balance:  Best Threshold: 0.6237 ; Best Accuracy: 0.7458  

In [ ]:
print('threshold 0.5328') 
threshold = 0.5328 

predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

print('threshold 0.6237') 
threshold = 0.6237
predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1score, _ = precision_recall_fscore_support(y_test_s, predictions_adj, average=None)

metrics_dict = {}
for i in range(len(precision)):
    class_label = f"class_{i-1}"
    metrics_dict[f"{class_label}_precision"] = round(precision[i],3)
    metrics_dict[f"{class_label}_recall"] = round(recall[i],3)
    metrics_dict[f"{class_label}_f1score"] = round(f1score[i],3)

print(metrics_dict)